In [ ]:
import numpy as np
import numpy.polynomial.polynomial as poly
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
from scipy.optimize import curve_fit
import warnings
import json
from wafer_plot import draw_wafer_plot

warnings.filterwarnings('ignore')

In [ ]:
def sin_func(x, A, P_pi, phi):
    return A * np.sin(2 * np.pi * x / P_pi + phi) + A

In [ ]:
root_path = Path('/Users/jiahaozhan/Google Drive/Other Computers/CM300/Dione_wafer3_BO')
data_path = root_path / 'pdk_thermphase_test_cells'

no = 3
data_dir_gen = sorted(data_path.glob(f'reticle27/*.json'))
for d in data_dir_gen:
    df = pd.read_json(d)
    # print(df.columns)
    # print(df['wavelength'])
    # ret = int(df['reticle'][0])
    
    wavel = df['wavelength'][no]
    trans = df['transmission_ch1'][no]
    volts = df['smu_voltage'][no]
    currs = df['smu_current'][no]

currs = [i - currs[0] for i in currs] # current should be 0 at 0 V
currs_mA = np.multiply(currs, 1000)

elec_pwrs = np.multiply(volts, currs_mA)
res = np.divide(volts[1:], currs[1:])

print(volts)
print(currs_mA)
print(res)

In [ ]:
# fit the transmission data at 1310 nm with a sinusoidal function
trans_1310 = [tr[1000] for tr in trans]
trans_1310_mW = 10**np.divide(trans_1310, 10)

x_data, y_data = elec_pwrs[:11], trans_1310_mW[:11]
x_data_new = np.arange(0, 50, 0.1)

# Guess the initial parameters
A_init = max(y_data)/2
P_init = 2*(abs(x_data[np.where(y_data == max(y_data))][0] - x_data[np.where(y_data == min(y_data))][0]))
phi_init = np.arcsin(y_data[0]/A_init - 1)
p_init = [A_init, 20, phi_init]
print(p_init)

p_opt, p_cov = curve_fit(sin_func, x_data, y_data, p0=p_init)
print(p_opt)

y_predicted = sin_func(x_data, *p_opt)
residuals = y_data - y_predicted
ss_total = np.sum((y_data - np.mean(y_data))**2)
ss_residual = np.sum(residuals**2)
r_squared = 1 - (ss_residual / ss_total)

fig, axes = plt.subplots(2, 1, figsize=(7, 10))
ax = axes[0]
# ax.plot(x_data, y_data, 'o')
ax.plot(elec_pwrs, trans_1310_mW, 'o')
ax.plot(x_data_new, sin_func(x_data_new, *p_opt))
ax.set_xlabel('Electrical power (mW)')
ax.set_ylabel('Optical power (mW)')
ax.set_title(title)
ax.set_xlim([0, 35])
ax.text(4, 0, f'$R^{2}$ = {r_squared:.4f}' + '\n' + '$P_{\pi}$ = ' + f'{p_opt[1]:.2f} mW', fontsize=12)

ax = axes[1]
ep_data, res_data = elec_pwrs[1:], res[:]
coefs = poly.polyfit(ep_data, res_data, 1)
ffit = poly.Polynomial(coefs) 
ax.plot(ep_data, res_data, 'o')
ax.plot(ep_data, ffit(ep_data))
ax.set_xlabel('Electrical power (mW)')
ax.set_ylabel('Heater resistance (ohm)')